In [7]:
import warnings
warnings.filterwarnings("ignore")

import os
import itertools

import torch

from typing import Any, Dict, List, Set

from detectron2.data import build_detection_train_loader
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch
from detectron2.evaluation import COCOEvaluator
from detectron2.solver.build import maybe_add_gradient_clipping

from ditod import add_vit_config
from ditod import DetrDatasetMapper

from detectron2.data.datasets import register_coco_instances
import logging
from detectron2.utils.logger import setup_logger
from detectron2.utils import comm
from detectron2.engine.defaults import create_ddp_model
import weakref
from detectron2.engine.train_loop import AMPTrainer, SimpleTrainer
from ditod import MyDetectionCheckpointer, ICDAREvaluator
from ditod import MyTrainer
from collections import defaultdict

In [8]:
from detectron2.utils.events import EventWriter, get_event_storage, has_event_storage, EventStorage
from collections import defaultdict

# def aim_track(aim_run, loss_dict, epoch):
#         # aim_run.track(items, epoch=epoch, context={'subset': 'train'})

#         # aim - Track weights and gradients distributions
#         # track_params_dists(model, aim_run)
#         # track_gradients_dists(model, aim_run)
#         # remainder.
#         # if i % 300 == 0:
#         logging.info("LOGS. Epoch:", epoch)
#         aim_run.track(
#             loss_dict, name=None, epoch=epoch, context={'subset': 'train', }
#         )
#         # aim_run.track(
#         #     acc, name='accuracy', epoch=epoch, context={'subset': 'val'}
#         # )
# class AIMWriter(EventWriter):
#     """
    
#     """

#     def __init__(self, json_file = "file.json", window_size=20):
#         """
#         Args:
#             json_file (str): path to the json file. New data will be appended if the file exists.
#             window_size (int): the window size of median smoothing for the scalars whose
#                 `smoothing_hint` are True.
#         """
#         # self._file_handle = PathManager.open(json_file, "a")
#         self._window_size = window_size
#         self._last_write = -1
#         print("INITIALIZE")

#     def write(self):
#         storage = get_event_storage()
#         print("Storage",storage)
#         to_save = defaultdict(dict)
#         print("TO SAVE", to_save)

#         for k, (v, iter) in storage.latest_with_smoothing_hint(self._window_size).items():
#             # keep scalars that have not been written
#             print("ITER", iter)
#             if iter <= self._last_write:
#                 continue
#                 print("LAST WRITE")
#             to_save[iter][k] = v
#         if len(to_save):
#             all_iters = sorted(to_save.keys())
#             self._last_write = max(all_iters)

#         for itr, scalars_per_iter in to_save.items():
#             scalars_per_iter["iteration"] = itr
#             # self._file_handle.write(json.dumps(scalars_per_iter, sort_keys=True) + "\n")
#             print("ITR", itr)
#             aim_track(aim_run, scalars_per_iter, itr)
        
# __all__ = [
#     "get_event_storage",
#     "has_event_storage",
#     "JSONWriter",
#     "TensorboardXWriter",
#     "CommonMetricPrinter",
#     "EventStorage",
#     "AIMWriter"
# ]


In [3]:
# from aim import Run
# # # Initialize a new Run
# aim_run = Run(repo = "/aim")


In [4]:
# from aim import Run

In [9]:
def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    # add_coat_config(cfg)
    add_vit_config(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    cfg.freeze()
    default_setup(cfg, args)
    return cfg


def main(args):
    """
    register publaynet first
    """
    register_coco_instances(
        "train_data",
        {},
        "./dataset/train.json",
        "./dataset/train"
    )

    register_coco_instances(
        "val_data",
        {},
        "./dataset/val.json",
        "./dataset/val"
    )
    args.eval_only = False
    cfg = setup(args)

    if args.eval_only:
        model = MyTrainer.build_model(cfg)
        DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
            cfg.MODEL.WEIGHTS, resume=args.resume
        )
        res = MyTrainer.test(cfg, model)
        return res

    trainer = MyTrainer(cfg)
    trainer.resume_or_load(resume=args.resume)
    return trainer.train()
    

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import argparse
import sys
parser = default_argument_parser()
parser.add_argument("--debug", action="store_true", help="enable debug mode")
# args = parser.parse_args()
command_args = "--config-file model/cascade_dit_base.yaml --num-gpus 1 MODEL.WEIGHTS model/model_weights.pth OUTPUT_DIR output/"
args = default_argument_parser().parse_args(command_args.split())

print("Command Line Args:", args)

# if args.debug:
#     import debugpy

#     print("Enabling attach starts.")
#     debugpy.listen(address=('0.0.0.0', 9310))
#     debugpy.wait_for_client()
#     print("Enabling attach ends.")

launch(
    main,
    args.num_gpus,
    num_machines=args.num_machines,
    machine_rank=args.machine_rank,
    dist_url=args.dist_url,
    args=(args,),
)
!cp "output/model_final.pth" "../model.pth"

Command Line Args: Namespace(config_file='model/cascade_dit_base.yaml', resume=False, eval_only=False, num_gpus=1, num_machines=1, machine_rank=0, dist_url='tcp://127.0.0.1:50152', opts=['MODEL.WEIGHTS', 'model/model_weights.pth', 'OUTPUT_DIR', 'output/'])
[01/15 16:26:27 detectron2]: Rank of current process: 0. World size: 1
[01/15 16:26:28 detectron2]: Environment info:
-------------------------------  -------------------------------------------------------------------------------------------------
sys.platform                     linux
Python                           3.10.13 | packaged by conda-forge | (main, Dec 23 2023, 15:36:39) [GCC 12.3.0]
numpy                            1.26.3
detectron2                       0.6 @/home/jovyan/work/detectron2/detectron2
Compiler                         GCC 11.4
CUDA compiler                    CUDA 12.2
detectron2 arch flags            /home/jovyan/work/detectron2/detectron2/_C.cpython-310-x86_64-linux-gnu.so; cannot find cuobjdump
DETECTRON

2024-01-15 16:26:54.064840: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 16:26:54.216171: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-15 16:26:54.221277: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-01-15 16:26:54.221293: I tensorflow/stream_ex

[01/15 16:27:17 d2.utils.events]:  eta: 4:50:33  iter: 39  total_loss: 3.673  loss_cls_stage0: 0.4245  loss_box_reg_stage0: 0.06443  loss_cls_stage1: 0.3385  loss_box_reg_stage1: 0.02939  loss_cls_stage2: 0.3549  loss_box_reg_stage2: 0.01338  loss_mask: 0.6661  loss_rpn_cls: 0.5917  loss_rpn_loc: 1.025    time: 1.0768  last_time: 0.7135  data_time: 0.0034  last_data_time: 0.0034   lr: 1.9326e-05  max_mem: 6021M
[01/15 16:27:39 d2.utils.events]:  eta: 4:56:23  iter: 59  total_loss: 2.922  loss_cls_stage0: 0.4171  loss_box_reg_stage0: 0.3973  loss_cls_stage1: 0.2419  loss_box_reg_stage1: 0.2066  loss_cls_stage2: 0.1568  loss_box_reg_stage2: 0.07309  loss_mask: 0.4802  loss_rpn_cls: 0.4106  loss_rpn_loc: 0.4675    time: 1.0811  last_time: 0.8155  data_time: 0.0032  last_data_time: 0.0031   lr: 2.7185e-05  max_mem: 6021M
[01/15 16:28:02 d2.utils.events]:  eta: 5:10:54  iter: 79  total_loss: 3.004  loss_cls_stage0: 0.377  loss_box_reg_stage0: 0.5853  loss_cls_stage1: 0.259  loss_box_reg_sta

In [ ]:
# !wget https://docs.google.com/uc?export=download&confirm=t&id=1_-3YRQKEIsreq_SyE1cOkcpvDRyRJORd